In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
sys.path.append('/kaggle/input/mnist_data')

In [ ]:
class Linear:
    def __init__(self, dim_in, dim_out):
        self.weights = np.random.randn(dim_in, dim_out) * 0.01
        self.biases = np.zeros((1, dim_out))
    
    def forward(self, input):
        self.input = input
        return (np.dot(input * self.weights) + self.biases)   

    def backward(self, gradient):
        self.weights.grad = np.dot(self.input.T, gradient)
        self.biases.grad = np.sum(gradient, axis = 0)
        return np.dot(gradient, self.weights.T)

    
class ReLU:
    def forward(self, input):
        self.input = input
        return np.maximum(0, input)
    def backward(self):
        return (self.input > 0)
    
class Softmax:
    def forward(self, x):
        self.output = np.exp(x - np.max(x, axis =1)) / np.sum(np.exp(x - np.max(x, axis =1)), axis = 1)
        return self.output

    def backward(self, y):
        return self.output - y

class CrossEntropyLoss:
    def forward(self, y, y_pred):
        self.ypred = y_pred
        self.y = y
        return -np.sum(y * np.log(y_pred))
    
    def backward(self):
        return self.ypred - self.y
    

class SGD:
  def __init__(self, learning_rate):
    self.learning_rate = learning_rate

  def update(self, model):
    for layer in model.layers:
      if hasattr(layer, 'weights'):
        layer.weights -= self.learning_rate * layer.weights.grad
        layer.biases -= self.learning_rate * layer.biases.grad
        

class Model:
    def __init__(self): 
        self.layers = []
    
    def add(self, layer):
        self.layers.append(layer)
    
    def compile(self, loss, optimizer):
        self.loss = loss  
        self.optimizer = optimizer
    
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, gradient):
      self.gradient = gradient
      for layer in reversed(self.layers):
        gradient = layer.backward(gradient)
        
      return gradient

    def train(self, x_train, y_train, epochs, batch_size):
        for epoch in range(epochs):
            for i in range(0, len(x_train), batch_size):
                x_batch = x_train[i:i + batch_size]
                y_batch = y_train[i:i + batch_size]

    def evaluate(self, x_test, y_test):
        predictions = self.forward(x_test)
        loss = self.loss.forward(y_test, predictions)
        accuracy = np.mean(predictions == y_test)
        return loss, accuracy

In [ ]:
# Define a simple neural network using the framework
model = Model()
model.add_layer(Linear(784, 128))
model.add_layer(ReLU())
model.add_layer(Linear(128, 10))
model.add_layer(Softmax())

# Compile the model with loss and optimizer
loss = CrossEntropyLoss()
optimizer = SGD(learning_rate=0.01)
model.compile(loss, optimizer)

# Assume x_train, y_train, x_test, y_test are preprocessed and available
# Train the model
model.train(x_train, y_train, epochs=20, batch_size=64)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')